In [1]:
import csv
import math

import numpy as np

### Tensorflow
import tensorflow as tf
from tensorflow.contrib import slim

In [2]:
INPUT_SIZE = 28
SPLIT_SIZE = 4

### train_x = train_x / NORMALIZE
NORMALIZE = 1.0
# NORMALIZE = 255   

In [3]:
### Dataset class
class Dataset(object) :
    
    def __init__(self, path, batch_size, shuffle = True, normalize = 255.0) :
        self.batch_size = batch_size
        self.shuffle    = shuffle
        self.normalize  = normalize
        self.xs, self.ys = self._load_data(path)
        
        self.data_size = self.xs.shape[0]
        self.batch_max = int(math.ceil(self.data_size / float(self.batch_size)))
        
    def _load_data(self, path) :
        reader = csv.reader(open(path), lineterminator = "\n")
        data   = []
        labels = []
        for row in reader :
            tmp = np.asarray([float(x) for x in row])
            
            ### Normalize
            tmp = tmp / self.normalize
            
            data.append(tmp[:INPUT_SIZE*INPUT_SIZE])
            labels.append(tmp[INPUT_SIZE*INPUT_SIZE:])
        
        data   = np.asarray(data)
        labels = np.asarray(labels)
        return (data, labels)

    def __call__(self) :
        indexes = None
        if self.shuffle :
            indexes = np.random.permutation(self.data_size)
        else :
            indexes = np.arange(self.data_size)

        for d_idx in range(self.batch_max) :
            start =     (d_idx + 0) * self.batch_size
            end   = min((d_idx + 1) * self.batch_size, self.data_size)
            
            batch_data_size = len(indexes[start:end])
            xs = np.zeros((batch_data_size, self.xs.shape[1]), dtype = np.float32)
            ys = np.zeros((batch_data_size, self.ys.shape[1]), dtype = np.float32)
            
            for b_idx, r_idx in enumerate(indexes[start:end]) :
                xs[b_idx] = self.xs[r_idx]
                ys[b_idx] = self.ys[r_idx]

            yield (xs, ys)

In [4]:
### Load dataset
dataset = Dataset("training_data_4x4.csv", 10, shuffle = True, normalize = NORMALIZE)

In [5]:
valid_dataset = Dataset("training_data_4x4.csv", 10, shuffle = False, normalize = NORMALIZE)

In [6]:
### Load batch data sample code
# for batch_num, (train_x, train_y) in enumerate(dataset(), 1) :
#     print("Batch {0}".format(batch_num))
#     print("Train shape : {0}".format(train_x.shape))

In [7]:
### for jupyter, interactive session.
sess = tf.InteractiveSession()

In [9]:
### Network settings
NN_INPUT_SIZE  = INPUT_SIZE * INPUT_SIZE
NN_OUTPUT_SIZE = 4 * SPLIT_SIZE

x  = tf.placeholder(tf.float32, shape=[None, NN_INPUT_SIZE])
y_ = tf.placeholder(tf.float32, shape=[None, NN_OUTPUT_SIZE])

def print_shape(layer) :
    print("Layer \"{0}\" shape : {1}".format(layer.name, layer.shape))

with slim.arg_scope([slim.conv2d, slim.fully_connected], activation_fn=tf.nn.relu,
    weights_initializer=tf.truncated_normal_initializer(stddev=0.1),
    biases_initializer=tf.constant_initializer(0.1)):
    with slim.arg_scope([slim.max_pool2d], padding='SAME'):
        x_image = slim.array_ops.reshape(x, [-1, INPUT_SIZE, INPUT_SIZE, 1])
        h_conv1 = slim.conv2d(x_image, 32, [5, 5])
        h_pool1 = slim.max_pool2d(h_conv1, [2, 2])

        h_conv2 = slim.conv2d(h_pool1, 32, [5, 5])
        h_pool2 = slim.max_pool2d(h_conv2, [2, 2])
        
        h_pool2_flat = slim.flatten(h_pool2)
        h_fc1 = slim.fully_connected(h_pool2_flat, 500)

        keep_prob = tf.placeholder(tf.float32)
        h_fc1_drop = slim.dropout(h_fc1, keep_prob)

        y_out  = slim.fully_connected(h_fc1_drop, NN_OUTPUT_SIZE, activation_fn=None)
        
print_shape(h_conv1)
print_shape(h_pool1)
print_shape(h_conv2)
print_shape(h_pool2)
print_shape(h_fc1)
print_shape(y_out)

Layer "Conv_3/Relu:0" shape : (?, 28, 28, 32)
Layer "MaxPool2D_3/MaxPool:0" shape : (?, 14, 14, 32)
Layer "Conv_4/Relu:0" shape : (?, 14, 14, 32)
Layer "MaxPool2D_4/MaxPool:0" shape : (?, 7, 7, 32)
Layer "fully_connected_2/Relu:0" shape : (?, 500)
Layer "fully_connected_3/BiasAdd:0" shape : (?, 16)


In [10]:
### Loss
loss = tf.losses.mean_squared_error(y_, y_out)

In [11]:
### Training step (Optimizer)
train_step = tf.train.AdamOptimizer(1e-4).minimize(loss)

In [12]:
### Initialize session
sess.run(tf.global_variables_initializer())

In [13]:
### Training
EPOCH = 10000
VALIDATION_EPOCH = 1000
for epoch in range(1, EPOCH + 1) :
    
    train_loss = 0
    for train_x, train_y in dataset() :
        _, tmp_loss = sess.run([train_step, loss], feed_dict = {x: train_x, y_: train_y, keep_prob: 0.5})
        batch_loss = tmp_loss / train_x.shape[0]
        train_loss += batch_loss

    ### Validation
    if epoch % VALIDATION_EPOCH == 0 :
        valid_loss = 0
        for (valid_x, valid_y) in valid_dataset() :
            _tmp_loss  = sess.run(loss, feed_dict = {x: train_x, y_: train_y, keep_prob: 1.0})
            batch_loss = _tmp_loss / valid_x.shape[0]
            valid_loss += batch_loss
        print("Epoch {0} train loss = {1}, valid loss = {2}, sqrt(valid loss) * {4} = {3}".format(epoch, train_loss, valid_loss, math.sqrt(valid_loss) * NORMALIZE, NORMALIZE))

Epoch 1000 train loss = 866.3181213378907, valid loss = 96.42523422241212, sqrt(valid loss) * 1.0 = 9.819635136929076
Epoch 2000 train loss = 625.6139953613281, valid loss = 73.51789245605468, sqrt(valid loss) * 1.0 = 8.574257545470317
Epoch 3000 train loss = 500.0207244873047, valid loss = 39.24505615234375, sqrt(valid loss) * 1.0 = 6.264587468648175
Epoch 4000 train loss = 476.5160247802734, valid loss = 71.26169471740722, sqrt(valid loss) * 1.0 = 8.441664214916822
Epoch 5000 train loss = 418.0080337524414, valid loss = 61.285160827636716, sqrt(valid loss) * 1.0 = 7.82848394183936
Epoch 6000 train loss = 481.97813720703124, valid loss = 30.08160972595215, sqrt(valid loss) * 1.0 = 5.484670430021493
Epoch 7000 train loss = 359.5241729736328, valid loss = 24.081302452087403, sqrt(valid loss) * 1.0 = 4.9072703667199145
Epoch 8000 train loss = 466.69944152832034, valid loss = 26.304337596893312, sqrt(valid loss) * 1.0 = 5.128775448086348
Epoch 9000 train loss = 436.4885925292969, valid lo

In [31]:
### Save session
saver = tf.train.Saver()
saver.save(sess, "cae_model.ckpt")

'cae_model.ckpt'

In [32]:
### Load session
saver.restore(sess, "cae_model.ckpt")

INFO:tensorflow:Restoring parameters from cae_model.ckpt
